# Day 4 - Lab 2: Generating a CI/CD Pipeline

**Objective:** Use an LLM to generate all necessary configuration files to create an automated Continuous Integration (CI) pipeline for the FastAPI application using Docker and GitHub Actions.

**Estimated Time:** 75 minutes

**Introduction:**
A robust CI pipeline is the backbone of modern software development. It automatically builds and tests your code every time a change is made, catching bugs early and ensuring quality. In this lab, you will generate all the configuration-as-code artifacts needed to build a professional CI pipeline for our application.

For definitions of key terms used in this lab, please refer to the [GLOSSARY.md](../../GLOSSARY.md).

## Step 1: Setup

We will load our application code to provide context for the LLM. The AI needs to see our code's imports to generate an accurate `requirements.txt` file.

**Model Selection:**
Models that are good at understanding code and structured data formats like YAML are ideal. `gpt-4.1`, `o3`, or `codex-mini` are strong choices.

**Helper Functions Used:**
- `setup_llm_client()`: To configure the API client.
- `get_completion()`: To send prompts to the LLM.
- `load_artifact()`: To read our application's source code.
- `save_artifact()`: To save the generated configuration files.
- `clean_llm_output()`: To clean up the generated text and code.

In [2]:
import sys
import os

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, save_artifact, load_artifact, clean_llm_output

client, model_name, api_provider = setup_llm_client(model_name="gemini-2.5-pro")

# Load the application code from Day 3 to provide context
app_code = load_artifact("app/main.py", base_dir=project_root)
if not app_code:
    print("Warning: Could not load app/main.py. Lab may not function correctly.")

2025-10-31 13:21:51,410 ag_aisoftdev.utils INFO LLM Client configured provider=google model=gemini-2.5-pro latency_ms=None artifacts_path=None


## Step 2: The Challenges

### Challenge 1 (Foundational): Generating a `requirements.txt`

**Task:** Before we can build a Docker image, we need a list of our Python dependencies. Prompt the LLM to analyze your application code and generate a `requirements.txt` file.

**Instructions:**
1.  Write a prompt that provides the LLM with the source code of your FastAPI application (`app_code`).
2.  Instruct it to analyze the `import` statements and generate a list of all external dependencies (like `fastapi`, `uvicorn`, `sqlalchemy`). You should also ask it to include `pytest` for testing.
3.  The output should be formatted as a standard `requirements.txt` file.
4.  Save the artifact to the project's root directory.

In [4]:
# Prompt to generate a requirements.txt file based on the loaded FastAPI app source code.
requirements_prompt = f"""
You are an assistant that extracts Python package dependencies from source code and outputs ONLY a valid requirements.txt file.

Context:
The following is the FastAPI application source code for analysis:
--- BEGIN APP CODE ---
{app_code}
--- END APP CODE ---

Instructions:
1. Read the import statements and determine all external (non-standard-library) Python packages required.
2. Include typical FastAPI stack dependencies if referenced or implied: fastapi, uvicorn[standard], pydantic, sqlalchemy, python-dotenv.
3. Include testing and tooling dependencies we will use in CI: pytest, pytest-asyncio.
4. Pin each dependency to a stable, recent version (use widely adopted latest minor/patch; avoid alpha/beta).
5. If SQLAlchemy appears, also include alembic if migrations seem plausible; otherwise omit.
6. Do NOT include packages that are part of the Python standard library (e.g., os, sys, typing, datetime, json, asyncio).
7. Output MUST be plain text in canonical requirements.txt format: one package per line, optionally version pinned with ==, no comments, no explanatory prose, no code fences.
8. Prefer the following ordering heuristic: core framework (fastapi) first, server (uvicorn), data/model packages (pydantic, sqlalchemy), utilities, testing packages last.
9. Avoid duplications; ensure consistent version pinning.
10. If you are uncertain about a package, make a reasonable assumption based on typical FastAPI CRUD apps using a database.

Output Format:
requirements.txt lines only. Example pattern (do not reuse exact versions blindly):
fastapi==<version>
uvicorn[standard]==<version>
...
pytest==<version>
pytest-asyncio==<version>

Return ONLY the lines. No commentary.
"""

print("--- Generating requirements.txt ---")
if app_code:
    requirements_content = get_completion(requirements_prompt, client, model_name, api_provider)
    cleaned_reqs = clean_llm_output(requirements_content, language='text')
    print(cleaned_reqs)
    # Backup existing file handled externally; now overwrite with new generated requirements.txt
    save_artifact(cleaned_reqs, "requirements.txt", base_dir=project_root, overwrite=True)
else:
    print("Skipping requirements generation because app code is missing.")

--- Generating requirements.txt ---
fastapi==0.111.0
uvicorn[standard]==0.30.1
pydantic==2.7.4
SQLAlchemy==2.0.31
alembic==1.13.1
python-dotenv==1.0.1
pytest==8.2.2
pytest-asyncio==0.23.7
fastapi==0.111.0
uvicorn[standard]==0.30.1
pydantic==2.7.4
SQLAlchemy==2.0.31
alembic==1.13.1
python-dotenv==1.0.1
pytest==8.2.2
pytest-asyncio==0.23.7


### Challenge 2 (Intermediate): Generating a `Dockerfile`

**Task:** Generate a multi-stage `Dockerfile` to create an optimized and secure container image for our application.

> **Tip:** Why a multi-stage Dockerfile? The first stage (the 'builder') installs all dependencies, including build-time tools. The final stage copies only the application code and the necessary installed packages. This results in a much smaller, more secure production image because it doesn't contain any unnecessary build tools.

**Instructions:**
1.  Write a prompt asking for a multi-stage `Dockerfile` for a Python FastAPI application.
2.  Specify the following requirements:
    * Use a slim Python base image (e.g., `python:3.11-slim`).
    * The first stage should install dependencies from `requirements.txt`.
    * The final stage should copy the installed dependencies and the application code.
    * The `CMD` should execute the application using `uvicorn`.
3.  Save the generated file as `Dockerfile` in the project's root.

In [7]:
# Prompt to generate a production-ready multi-stage Dockerfile for the FastAPI app.
dockerfile_prompt = f"""
You are an expert DevOps assistant. Generate ONLY a valid multi-stage Dockerfile (no backticks, no commentary) for a FastAPI application.

Context Code (for imports & structure reference):
--- BEGIN APP CODE ---
{app_code}
--- END APP CODE ---

Requirements:
1. Use a builder stage based on python:3.11-slim (name it builder).
2. Create and activate a virtual environment in /opt/venv OR use --prefix to install dependencies into /opt/venv; ensure final image copies this environment.
3. Copy in requirements.txt and install dependencies with: pip install --no-cache-dir -r requirements.txt.
4. Separate dev/test deps if a requirements-dev.txt exists (conditionally) – but assume single file if not mentioned.
5. Perform security-related best practices: 
   - RUN apt-get update && apt-get install -y build-essential (only if needed for wheels) then apt-get purge -y --auto-remove build-essential && rm -rf /var/lib/apt/lists/*.
   - Use pip install flags: --no-cache-dir, and consider --upgrade pip before installs.
6. Final stage:
   - Base image: python:3.11-slim.
   - Copy the virtual environment from builder.
   - Copy the application code (app/ and any root entrypoint like main.py if needed).
   - Set ENV PATH=/opt/venv/bin:$PATH.
   - Create a non-root user (e.g., appuser) and switch to it.
   - Expose port 8000.
7. Entrypoint/CMD should run uvicorn with host 0.0.0.0 and port 8000. If main app is in app/main.py and FastAPI instance named app, use: uvicorn app.main:app --host 0.0.0.0 --port 8000.
8. Multi-stage should reduce final image size and not include build tools after build.
9. Include HEALTHCHECK (optional) hitting /docs or /openapi.json with curl.
10. No extraneous comments except minimal stage identifiers; no placeholder text.
11. Do NOT reference files not guaranteed to exist (avoid requirements-dev.txt unless conditional). Keep deterministic.
12. Avoid ARGs unless necessary; focus on clarity.
13. Ensure reproducibility and security (no root long-term, minimal layers).

Output: Only the Dockerfile contents.
"""

print("--- Generating Dockerfile ---")
dockerfile_content = get_completion(dockerfile_prompt, client, model_name, api_provider)
cleaned_dockerfile = clean_llm_output(dockerfile_content, language='dockerfile')
print(cleaned_dockerfile)

if cleaned_dockerfile:
    # Ensure we write to repository root, not artifacts subdirectory
    save_artifact(cleaned_dockerfile, "Dockerfile", base_dir=project_root, overwrite=True)

--- Generating Dockerfile ---
# syntax=docker/dockerfile:1

FROM python:3.11-slim as builder
ENV PIP_NO_CACHE_DIR=off
ENV PIP_DISABLE_PIP_VERSION_CHECK=on
RUN python -m pip install --upgrade pip
WORKDIR /app
COPY requirements.txt .
RUN pip install --no-cache-dir --prefix=/opt/venv -r requirements.txt

FROM python:3.11-slim
ENV PYTHONDONTWRITEBYTECODE=1
ENV PYTHONUNBUFFERED=1
ENV PATH=/opt/venv/bin:$PATH
RUN apt-get update \
    && apt-get install -y --no-install-recommends curl \
    && apt-get clean \
    && rm -rf /var/lib/apt/lists/*
RUN addgroup --system --gid 1001 appgroup && \
    adduser --system --uid 1001 --ingroup appgroup appuser
COPY --from=builder /opt/venv /opt/venv
WORKDIR /home/appuser/app
COPY --chown=appuser:appgroup app/ ./app/
USER appuser
EXPOSE 8000
HEALTHCHECK --interval=30s --timeout=5s --start-period=5s --retries=3 \
  CMD curl -f http://localhost:8000/ || exit 1
CMD ["uvicorn", "app.main:app", "--host", "0.0.0.0", "--port", "8000"]
# syntax=docker/dockerfile:1

### Bonus: Consider a `.dockerignore`
A well-crafted `.dockerignore` keeps build contexts lean and secure. After generating the Dockerfile, you can prompt the LLM similarly to create a `.dockerignore` that excludes:
- __pycache__/ and *.pyc
- .git/, .github/
- .venv/ or any local virtual environments
- .DS_Store
- notebooks (`*.ipynb`) if not required at runtime
- tests/ (if you only run them in CI)
- large local artifact directories like `artifacts/`, `sandbox_runs/`, `slides/`

Prompt idea:
"""
Generate a .dockerignore file for a FastAPI app. Exclude caches, VCS metadata, virtual environments, notebooks, test directories, local artifacts, and OS cruft. Only output the .dockerignore lines.
"""

You can then save with:
```python
save_artifact(clean_llm_output(dockerignore_content, language='text'), '.dockerignore')
```

In [5]:
# Generate a .dockerignore file via LLM (optional enhancement)
dockerignore_prompt = """
You are to output ONLY .dockerignore patterns (one per line, no comments, no code fences).
Context: Python FastAPI application with directories: app/, utils/, tests/, async_tests/, artifacts/, sandbox_runs/, slides/, Labs/, Solutions/, Supporting Materials/.
Goal: Minimize image size and exclude development, test, artifact, and environment-specific files.

Include exclusions for:
- Python caches: __pycache__/ , *.pyc , *.pyo
- VCS & CI: .git/ , .github/
- Virtual environments: .venv/ , venv/
- Editor settings: .vscode/ , .idea/
- Environment files: .env , .env.*
- Notebooks: *.ipynb
- Artifacts & non-runtime data: artifacts/ , sandbox_runs/ , slides/ , Labs/ , Solutions/ , Supporting Materials/
- Tests: tests/ , async_tests/
- Build & cache output: .cache/ , build/ , dist/ , coverage/
- Logs: *.log
- DB/backups: *.db* , *.sqlite* , *.sql , *.backup*
- OS cruft: .DS_Store , Thumbs.db

Do NOT exclude: app/ , utils/ , requirements.txt , Dockerfile
Avoid duplicates. Output only patterns.
"""

print("--- Generating .dockerignore (optional) ---")
dockerignore_content = get_completion(dockerignore_prompt, client, model_name, api_provider)
cleaned_dockerignore = clean_llm_output(dockerignore_content, language='text')
print(cleaned_dockerignore)

if cleaned_dockerignore:
    save_artifact(cleaned_dockerignore, ".dockerignore", base_dir=project_root, overwrite=True)

--- Generating .dockerignore (optional) ---
__pycache__/
*.pyc
*.pyo
.git/
.github/
.venv/
venv/
.vscode/
.idea/
.env
.env.*
*.ipynb
artifacts/
sandbox_runs/
slides/
Labs/
Solutions/
Supporting Materials/
tests/
async_tests/
.cache/
build/
dist/
coverage/
*.log
*.db*
*.sqlite*
*.sql
*.backup*
.DS_Store
Thumbs.db
__pycache__/
*.pyc
*.pyo
.git/
.github/
.venv/
venv/
.vscode/
.idea/
.env
.env.*
*.ipynb
artifacts/
sandbox_runs/
slides/
Labs/
Solutions/
Supporting Materials/
tests/
async_tests/
.cache/
build/
dist/
coverage/
*.log
*.db*
*.sqlite*
*.sql
*.backup*
.DS_Store
Thumbs.db


### Challenge 3 (Advanced): Generating the GitHub Actions Workflow

**Task:** Generate a complete GitHub Actions workflow file to automate the build and test process.

**Instructions:**
1.  Write a prompt to generate a GitHub Actions workflow file named `ci.yml`.
2.  Specify the following requirements for the workflow:
    * It should trigger on any `push` to the `main` branch.
    * It should define a single job named `build-and-test` that runs on `ubuntu-latest`.
    * The job should have steps to: 1) Check out the code, 2) Set up a Python environment, 3) Install dependencies from `requirements.txt`, and 4) Run the test suite using `pytest`.
3.  Save the generated YAML file to `.github/workflows/ci.yml`.

In [6]:
# Prompt to generate a GitHub Actions CI workflow (ci.yml)
ci_workflow_prompt = f"""
You are an expert CI engineer. Generate ONLY a valid GitHub Actions workflow YAML (no backticks, no commentary) named ci.yml for a Python FastAPI project.

Repository Context:
- FastAPI app code located in app/.
- Dependencies listed in requirements.txt at repo root.
- Tests located in tests/ and async_tests/ (we will run both).
- Dockerfile at repo root for building container image.

Workflow Requirements:
1. Trigger: on push to main and on pull_request targeting main.
2. Concurrency: Cancel in-progress runs of same ref to reduce noise.
3. Single job: build-and-test on ubuntu-latest.
4. Steps (in order):
   a. Checkout code (use actions/checkout@v4 with fetch-depth: 0 for potential future coverage or versioning).
   b. Set up Python 3.11 (actions/setup-python@v5) and enable pip caching keyed on requirements.txt hash.
   c. Install dependencies: pip install --upgrade pip && pip install -r requirements.txt.
   d. Run tests (pytest) across tests/ and async_tests/ with verbose output; generate junit xml and coverage report.
   e. Upload test results (use actions/upload-artifact@v4) for junit XML and coverage data.
   f. (Optional) Build Docker image using Dockerfile to ensure it compiles: docker build -t fastapi-ci-test .
5. Environment variables: set PYTHONUNBUFFERED=1.
6. Fail fast if dependency install fails; use --no-cache-dir.
7. Add a matrix strategy placeholder commented out for future Python versions (but keep current single version for simplicity).
8. Use minimal permissions: contents: read.
9. Add caching for pip (actions/cache) if not using built-in setup-python caching, but prefer built-in cache option.
10. Do NOT include deployment steps; CI only.
11. Ensure test command exits non-zero on failures (no continue-on-error).
12. Coverage: produce coverage.xml via pytest --cov=app --cov-report xml and store artifact.

Output: ONLY the YAML body of .github/workflows/ci.yml.
"""

print("--- Generating GitHub Actions Workflow ---")
ci_workflow_content = get_completion(ci_workflow_prompt, client, model_name, api_provider)
cleaned_yaml = clean_llm_output(ci_workflow_content, language='yaml')
print(cleaned_yaml)

if cleaned_yaml:
    # Write workflow to repo root under .github/workflows
    save_artifact(cleaned_yaml, ".github/workflows/ci.yml", base_dir=project_root, overwrite=True)

--- Generating GitHub Actions Workflow ---
name: Python FastAPI CI

on:
  push:
    branches: [ "main" ]
  pull_request:
    branches: [ "main" ]

permissions:
  contents: read

concurrency:
  group: ${{ github.workflow }}-${{ github.ref }}
  cancel-in-progress: true

jobs:
  build-and-test:
    runs-on: ubuntu-latest
    # strategy:
    #   fail-fast: false
    #   matrix:
    #     python-version: ["3.9", "3.10", "3.11"]

    env:
      PYTHONUNBUFFERED: "1"

    steps:
      - name: Checkout repository
        uses: actions/checkout@v4
        with:
          fetch-depth: 0

      - name: Set up Python 3.11
        uses: actions/setup-python@v5
        with:
          python-version: '3.11' # ${{ matrix.python-version }}
          cache: 'pip'
          cache-dependency-path: 'requirements.txt'

      - name: Install dependencies
        run: |
          python -m pip install --upgrade pip
          pip install --no-cache-dir -r requirements.txt

      - name: Run tests with pytest


## Lab Conclusion

Excellent! You have now generated a complete, professional Continuous Integration pipeline using AI. You created the dependency list, the containerization configuration, and the automation workflow, all from simple prompts. This is a powerful demonstration of how AI can automate complex DevOps tasks, allowing teams to build and ship software with greater speed and confidence.

> **Key Takeaway:** AI is a powerful tool for generating 'Configuration as Code' artifacts. By prompting for specific formats like `requirements.txt`, `Dockerfile`, or `ci.yml`, you can automate the creation of the files that define your entire build, test, and deployment processes.